[一个几乎可以解决所有NLP问题的kernal](https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle)

*   tfidf
*count features
*logistic regression
*naive bayes
*svm
*xgboost
*grid search
*word vectors
*LSTM
*GRU
*Ensembling

所属kaggle比赛：

spooky author 分类

所用框架： tf


In [1]:

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p GuodongDrive
!google-drive-ocamlfuse GuodongDrive

In [5]:
import os
os.chdir("GuodongDrive")
os.chdir("Spooky Author Identification")
!ls

 sample_submission.csv		       test.csv
'Spooky Author Identification.ipynb'   train.csv


In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import numpy as np
import pandas as pd
import xgboost as xgb
from tqdm import tqdm #可扩展的Python进度条
from keras.models import Sequential
from sklearn.svm import SVC
from keras.layers.recurrent import LSTM,GRU
from keras.layers.core import Dense,Activation,Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

先看一下data长什么样

In [10]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [11]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [12]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


Kaggle采用[multi-class log-loss](https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne/util.py)作为预测指标

In [0]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

我们用scikitlearn中的LabelEncoder 将类别文本转化为0,1,2

In [0]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [15]:
print(y)

[0 1 0 ... 0 0 1]


在进行下一步特征处理之前，我建议将数据分为训练集和验证集

In [0]:
x_train,x_valid,y_train,y_valid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [17]:
print(x_train.shape)
print(x_valid.shape)

(17621,)
(1958,)


#建立基本模型

首先第一个是tf-idf +LR

**Always start with these features. They work (almost) everytime!**

In [18]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(x_train) + list(x_valid))
x_train_tfv =  tfv.transform(x_train) 
x_valid_tfv = tfv.transform(x_valid)
print(x_train_tfv)

  (0, 11813)	0.2469424385443994
  (0, 10050)	0.31534173991151543
  (0, 7734)	0.2621357570531394
  (0, 6078)	0.21913481148346806
  (0, 5920)	0.2699803359499066
  (0, 5641)	0.28210670387497144
  (0, 3647)	0.33928338965309934
  (0, 3312)	0.2913532806410929
  (0, 2813)	0.3345950093082599
  (0, 2161)	0.3474674493892856
  (0, 1489)	0.3710563608683541
  (1, 14862)	0.3467364483300827
  (1, 11446)	0.23785570257025115
  (1, 9156)	0.3385709613222204
  (1, 8899)	0.49640563902064305
  (1, 8268)	0.4364136618558541
  (1, 4746)	0.3259827441631435
  (1, 72)	0.4067284254948717
  (2, 14071)	0.3240192548729904
  (2, 13042)	0.25915485523649134
  (2, 12014)	0.26382082678856383
  (2, 11546)	0.16409370088652775
  (2, 9656)	0.25408177755421213
  (2, 9422)	0.26141001709560185
  (2, 7995)	0.28804138618156144
  :	:
  (17617, 2312)	0.3765818334354421
  (17618, 14413)	0.41062696740663335
  (17618, 13333)	0.41457868692900307
  (17618, 12513)	0.2857797208393344
  (17618, 8238)	0.3745429502598771
  (17618, 3074)	0.384

In [19]:
print(x_train_tfv.shape)

(17621, 15102)


In [20]:
#用简单的LR模型试一下
clf = LogisticRegression(C=1.0)
clf.fit(x_train_tfv, y_train)
predictions = clf.predict_proba(x_valid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


logloss: 0.626 


**为了提高准确性，我们可以采用其他特征，比如统计词数，采用scikitlearn中的CountVectorizer**

In [0]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(x_train) + list(x_valid))
x_train_ctv =  ctv.transform(x_train) 
x_valid_ctv = ctv.transform(x_valid)

我们只提高了0.1.......
现在我们继续再用贝叶斯来试一下

In [22]:
clf= MultinomialNB()
clf.fit(x_train_tfv,y_train)
predictions = clf.predict_proba(x_valid_tfv)
print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.578 


看得出来还是LR更准确一点，但如果NB用在词数统计上呢

In [23]:
clf = MultinomialNB()
clf.fit(x_train_ctv, y_train)
predictions = clf.predict_proba(x_valid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.485 


所以可以看到RF仍然表现很好

另外一个ML 算法是SVM，但是SVM计算比较耗时，我们要减小特征数量

**对SVD来说，120-200个特征数量是比较合适的**

In [0]:
svd = decomposition.TruncatedSVD(n_components = 120)
svd.fit(x_train_tfv)
x_train_svd = svd.transform(x_train_tfv)
x_valid_svd = svd.transform(x_valid_tfv)
#归一化
scl = preprocessing.StandardScaler()
scl.fit(x_train_svd)
x_train_svd_scl = scl.transform(x_train_svd)
x_valid_svd_scl = scl.transform(x_valid_svd)

**接下来使用SVM**

In [0]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(x_train_svd_scl, y_train)
predictions = clf.predict_proba(x_valid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.740 


**看样子SVM表现性能并不好**

在进行下一步之前，我们尝试一下现在最主流的xgboost模型

In [25]:
clf = xgb.XGBClassifier(max_depth = 7,n_estimators = 200,colsample_bytree = 0.8,
                       subsample= 0.8,nthread = 10,learning_rate = 0.1)
clf.fit(x_train_tfv,y_train)
predictions = clf.predict_proba(x_valid_tfv)
#predictions = clf.predict_proba(x_valid_tfv.tocsc())
print ("logloss: %0.3f " % multiclass_logloss(y_valid, predictions))

logloss: 0.782 


看起来xgboost性能也不够好，但这是错误的，因为我们还没有进行超参数优化

#Grid Search

常用的超参数调试经验在个[参考链接中](http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/)

这里仅对LR中的参数进行调试

在进行之前，需要先建立评价函数，采用scikit-learn中的make——scorer

In [0]:
mll_scorer = metrics.make_scorer(multiclass_logloss,
                                 greater_is_better=False, needs_proba=True)

In [0]:
#初始化SVD
svd = TruncatedSVD()

scl = preprocessing.StandardScaler()

lr_model = LogisticRegression()

clf = pipeline.Pipeline([
    ('svd',svd),('scl',scl),('lr',lr_model)
])

In [0]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

In [38]:
clf.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'svd', 'scl', 'lr', 'svd__algorithm', 'svd__n_components', 'svd__n_iter', 'svd__random_state', 'svd__tol', 'scl__copy', 'scl__with_mean', 'scl__with_std', 'lr__C', 'lr__class_weight', 'lr__dual', 'lr__fit_intercept', 'lr__intercept_scaling', 'lr__l1_ratio', 'lr__max_iter', 'lr__multi_class', 'lr__n_jobs', 'lr__penalty', 'lr__random_state', 'lr__solver', 'lr__tol', 'lr__verbose', 'lr__warm_start'])

In [39]:
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(x_train_tfv, y_train)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   28.3s
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.2min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
 

Best score: -0.741
Best parameters set:
	lr__C: 1.0
	lr__penalty: 'l2'
	svd__n_components: 180


**接下来我们在navie bayes上试一下**

In [41]:
nb_model = MultinomialNB()

#建立pipeline
clf = pipeline.Pipeline([('nb', nb_model)])
param_grid = {'nb__alpha':[0.001,0.01,0.1,1,10,100]}
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)
model.fit(x_train_tfv, y_train)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Best score: -0.492
Best parameters set:
	nb__alpha: 0.1


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.2s finished


可以看到这有明显的提升

#Word Vectors

在这里我会简单介绍如何构造词向量和构建模型

在这里将会用到GloVe vectors

你可以从这个[参考链接](http://www-nlp.stanford.edu/data/glove.840B.300d.zip)中下载GloVe vectors

这里不再运行，因为作者并没有运行成功。。。